<h3>Imports</h3>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import glob, os
import random
import sklearn
import tensorflow as tf  
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, Input, Dropout, LSTM, LeakyReLU

<h3>Config Tensorflow to use GPU</h3>

In [2]:
config = tf.compat.v1.ConfigProto()  
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU  
config.log_device_placement = True  # to log device placement (on which device the operation ran)  
                                    # (nothing gets printed in Jupyter, only if you run it standalone)
sess = tf.compat.v1.Session(config=config)  
tf.compat.v1.keras.backend.set_session(sess)  # set this TensorFlow session as the default session for Keras  

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2080, pci bus id: 0000:08:00.0, compute capability: 7.5



<h3>Load raw data</h3>

In [3]:
lines = []

for file in glob.glob("C:\\Users\\Jordan\\Desktop\\data\\*.txt"):
    data_file = open(file)
    lines = lines + data_file.readlines()
    

<h3>Prepare training data</h3>

In [6]:
# each sample consists of 3 lines,
# Input state, output controls, output angle
num_samples = len(lines) // 3
seq_len = 200 # length of sequences to train on

# has one input and two outputs, one for probabilistic output and other for regression
data_x = []
data_y1 = []
data_y2 = []

# format data into input list and output lists
for i in range (0, num_samples * 3):
    if i % 3 == 0:
        data_x.append(np.fromstring(lines[i], dtype='float', sep=' '))
        data_y1.append(np.fromstring(lines[i + 1], dtype='float', sep=' '))
        data_y2.append(np.fromstring(lines[i + 2], dtype='float', sep=' '))

# format data for training
# has shape of (num sequences, seq length, number of features)
x_train = []
y1_train = []
y2_train = []

# create many sequences from a sliding window of 4 second sequences
i = 0
while i < num_samples - seq_len:
    seq_x = []
    seq_y1 = []
    seq_y2 = []
    
    for j in range (i, i + seq_len):
        seq_x.append(data_x[j])
        seq_y1.append(data_y1[j])
        seq_y2.append(data_y2[j])
        
    x_train.append(seq_x)
    y1_train.append(seq_y1)
    y2_train.append(seq_y2)
    
    i += random.randint(1, 10)
        
x_train = np.array(x_train)
y1_train = np.array(y1_train)
y2_train = np.array(y2_train)

num_features = x_train[0][0].size
output_size_y1 = y1_train[0][0].size
output_size_y2 = y2_train[0][0].size

print(x_train.shape)
print(y1_train.shape)
print(y2_train.shape)

(63902, 200, 36)
(63902, 200, 16)
(63902, 200, 2)


<h3>Create Model</h3>

In [7]:
new_model = False
checkpoint_path = "training_5\\cp.ckpt"

# creates the neural network
# the training model is not stateful and trains on 4 second sequences
# the predcition model is stateful and the internal state will persist between predictions
def createModel(for_training):
    if for_training:
        layer_input = Input(shape = (seq_len, num_features))
        is_stateful = False
    else:
        layer_input = Input(batch_shape = (1, 1, num_features))
        is_stateful = True
    
    layer_hid1 = LSTM(400, stateful=is_stateful, return_sequences=True)(layer_input)
    dropout = Dropout(0.2)(layer_hid1)
    layer_hid2 = LSTM(400, stateful=is_stateful, return_sequences=True)(dropout)
    dropout2 = Dropout(0.2)(layer_hid2)
    layer_hid3 = Dense(100)(dropout2)
    layer_hid3 = LeakyReLU(alpha=0.05)(layer_hid3)
    dropout3 = Dropout(0.2)(layer_hid3)
    
    # button outputs are probabilistic, goal angles are regressional
    layer_out1 = Dense(output_size_y1, activation = 'sigmoid', name='layer_out1')(dropout3)
    layer_out2 = Dense(output_size_y2, activation = 'linear', name='layer_out2')(dropout3)

    model = Model(inputs=layer_input, outputs=[layer_out1, layer_out2])

    if for_training:
        model.compile(
            loss={'layer_out1' : 'binary_crossentropy', 'layer_out2' : 'mean_squared_error'}, 
            loss_weights = {'layer_out1' : 8, 'layer_out2' : 0.5}, # 8, 0.5
            optimizer='adam', 
            metrics=['accuracy']
        )

    model.summary()
    
    return model

training_model = createModel(True)

if not new_model:
    training_model.load_weights(checkpoint_path)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 200, 36)]    0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     (None, 200, 400)     699200      input_1[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 200, 400)     0           lstm[0][0]                       
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 200, 400)     1281600     dropout[0][0]                    
______________________________________________________________________________________________

<h3>Train</h3>

In [ ]:

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

history = training_model.fit(x_train, {'layer_out1' : y1_train, 'layer_out2' : y2_train},
                    epochs=1000, batch_size = 256, shuffle=True,
                    callbacks=[cp_callback])


Train on 63902 samples
Epoch 1/1000
63744/63902 [============================>.] - ETA: 0s - loss: 48.1469 - layer_out1_loss: 0.2083 - layer_out2_loss: 92.9615 - layer_out1_accuracy: 0.9063 - layer_out2_accuracy: 0.9026
Epoch 00001: saving model to training_5/cp.ckpt
63902/63902 [==============================] - 40s 620us/sample - loss: 48.0750 - layer_out1_loss: 0.2082 - layer_out2_loss: 92.7299 - layer_out1_accuracy: 0.9063 - layer_out2_accuracy: 0.9027
Epoch 2/1000
63744/63902 [============================>.] - ETA: 0s - loss: 16.0254 - layer_out1_loss: 0.1863 - layer_out2_loss: 29.0695 - layer_out1_accuracy: 0.9176 - layer_out2_accuracy: 0.9387
Epoch 00002: saving model to training_5/cp.ckpt
63902/63902 [==============================] - 40s 621us/sample - loss: 16.0226 - layer_out1_loss: 0.1863 - layer_out2_loss: 29.0605 - layer_out1_accuracy: 0.9176 - layer_out2_accuracy: 0.9387
Epoch 3/1000
63744/63902 [============================>.] - ETA: 0s - loss: 12.8021 - layer_out1_loss

Epoch 20/1000
63744/63902 [============================>.] - ETA: 0s - loss: 5.3033 - layer_out1_loss: 0.0857 - layer_out2_loss: 9.2357 - layer_out1_accuracy: 0.9672 - layer_out2_accuracy: 0.9635
Epoch 00020: saving model to training_5/cp.ckpt
63902/63902 [==============================] - 40s 621us/sample - loss: 5.3044 - layer_out1_loss: 0.0857 - layer_out2_loss: 9.2394 - layer_out1_accuracy: 0.9672 - layer_out2_accuracy: 0.9635
Epoch 21/1000
63744/63902 [============================>.] - ETA: 0s - loss: 5.2338 - layer_out1_loss: 0.0836 - layer_out2_loss: 9.1294 - layer_out1_accuracy: 0.9681 - layer_out2_accuracy: 0.9640
Epoch 00021: saving model to training_5/cp.ckpt
63902/63902 [==============================] - 40s 631us/sample - loss: 5.2335 - layer_out1_loss: 0.0836 - layer_out2_loss: 9.1284 - layer_out1_accuracy: 0.9681 - layer_out2_accuracy: 0.9640
Epoch 22/1000
63744/63902 [============================>.] - ETA: 0s - loss: 5.1178 - layer_out1_loss: 0.0816 - layer_out2_loss: 8

Epoch 39/1000
63744/63902 [============================>.] - ETA: 0s - loss: 4.1039 - layer_out1_loss: 0.0627 - layer_out2_loss: 7.2044 - layer_out1_accuracy: 0.9764 - layer_out2_accuracy: 0.9674
Epoch 00039: saving model to training_5/cp.ckpt
63902/63902 [==============================] - 40s 630us/sample - loss: 4.1052 - layer_out1_loss: 0.0627 - layer_out2_loss: 7.2084 - layer_out1_accuracy: 0.9764 - layer_out2_accuracy: 0.9674
Epoch 40/1000
63744/63902 [============================>.] - ETA: 0s - loss: 4.1043 - layer_out1_loss: 0.0623 - layer_out2_loss: 7.2120 - layer_out1_accuracy: 0.9766 - layer_out2_accuracy: 0.9672
Epoch 00040: saving model to training_5/cp.ckpt
63902/63902 [==============================] - 40s 631us/sample - loss: 4.1042 - layer_out1_loss: 0.0623 - layer_out2_loss: 7.2116 - layer_out1_accuracy: 0.9766 - layer_out2_accuracy: 0.9672
Epoch 41/1000
63744/63902 [============================>.] - ETA: 0s - loss: 4.0262 - layer_out1_loss: 0.0612 - layer_out2_loss: 7

<h3>Create and save stateful prediction model</h3>

In [20]:
prediction_model = createModel(False)
prediction_model.load_weights(checkpoint_path)
prediction_model.save('mordhai.h5', include_optimizer=False)

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(1, 1, 36)]         0                                            
__________________________________________________________________________________________________
lstm_10 (LSTM)                  (1, 1, 400)          699200      input_6[0][0]                    
__________________________________________________________________________________________________
dropout_13 (Dropout)            (1, 1, 400)          0           lstm_10[0][0]                    
__________________________________________________________________________________________________
lstm_11 (LSTM)                  (1, 1, 400)          1281600     dropout_13[0][0]                 
____________________________________________________________________________________________